In [ ]:
import pandas as pd
import numpy as np

Reading the file

In [ ]:
lyrics = pd.read_csv("songsRecommendationSystem/Data/reducedfile.csv")

Filtering dataset

In [ ]:
filtered = lyrics[lyrics['artist'].str.startswith('Genius')].index
lyrics.drop(filtered, inplace=True)

In [ ]:
artistsFiltered = lyrics[lyrics['artist'].isin(['Genius Romanizations', 'Genius English Translations', 'Genius Brasil Tradues', 'Genius Traducciones al Espaol', 'Genius Traductions Franaises', 'Genius Trke eviri', 'Genius Russian Translations ( )', 'Genius Deutsche bersetzungen', 'Genius Traduzioni Italiane', 'Polskie tumaczenia Genius', 'Sports Genius', 'Rap Genius Deutschland','Genius Users','Spotify', 'Genius France', 'Genius', 'Genius Nederlandse Vertalingen', 'Unknown Artist', 'Genius Farsi Translations ( )', 'Genius Translations', 'Genius  (Chinese Translations)', 'Genius Srpski Prevodi', 'Genius Swedish Translations', 'Genius  (Japanese Translations)', 'Genius magyar fordtsok', 'Plylist', 'Genius India'])].index
lyrics.drop(artistsFiltered, inplace=True)

In [ ]:
yearFilter = lyrics[lyrics['year']<1800].index
lyrics.drop(yearFilter,inplace=True)

In [ ]:
lyrics['lyrics'] = lyrics['lyrics'].str.replace('​', '')
lyrics['artist'] = lyrics['artist'].str.replace('​', '')
lyrics['artist'] = lyrics['artist'].str.replace(' ', '')
lyrics['title'] = lyrics['title'].str.replace('​', '')
lyrics['lyrics'] = lyrics['lyrics'].str.replace('\n' , ' ')
lyrics['lyrics'] = lyrics['lyrics'].str.replace(' ', ' ')
lyrics.drop(lyrics[lyrics['title'] == 'nan'].index, inplace=True)
lyrics.drop(lyrics[lyrics['tag'] == 'misc'].index, inplace=True)

In [ ]:
lyrics['language'] = lyrics['language'].apply(lambda x: str(x))
lyrics['language_cld3'] = lyrics['language_cld3'].apply(lambda x: str(x))
lyrics['language_ft'] = lyrics['language_ft'].apply(lambda x: str(x))
nullLang = lyrics[lyrics['language_cld3'] == 'nan'].index
lyrics.drop(nullLang, inplace=True)
nullLang = lyrics[lyrics['language_ft'] == 'nan'].index
lyrics.drop(nullLang, inplace=True)

In [ ]:
lyrics['features'] = lyrics['features'].str.replace('{}', '')
lyrics['features'] = lyrics['features'].str.replace('{', '')
lyrics['features'] = lyrics['features'].str.replace('}', '')
lyrics['features'] = lyrics['features'].str.replace('​', '')
lyrics['features'] = lyrics['features'].str.replace('"', '')
lyrics['features'] = lyrics['features'].str.replace("'", '')

Preparing data for TFIDF

In [ ]:
lyrics['data'] = lyrics['artist'] + ' ' + lyrics['artist'] + ' ' + lyrics['artist'] + ' ' + lyrics['artist'] + ' ' + lyrics['title'] + ' ' + lyrics['lyrics'] + ' ' + lyrics['title'] + ' ' + lyrics['artist'] + ' ' + lyrics['tag'] + ' ' + lyrics['tag'] + ' ' + lyrics['tag'] + ' ' + lyrics['tag'] + ' ' + lyrics['tag']

In [ ]:
dataNull = lyrics[lyrics['data'].isnull()].index
lyrics.drop(dataNull, inplace=True)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

vectorizer = TfidfVectorizer()
tfidf = vectorizer.fit_transform(lyrics['data'])


In [ ]:
song1 = lyrics.iloc[39479]['data'] #any random song

newMatrix = vectorizer.transform([song1])

cosineSimilarities = cosine_similarity(newMatrix, tfidf)
cosineSimilarities

A function that takes an array of songs and returns the most similar ones

In [ ]:
def recommendation(data):

    #texts = lyrics[lyrics['id'].isin(IDS)]['data']

    # print(data)
    lst = []
    for text in data:

        new_text_matrix= vectorizer.transform([text])
        lst.append(new_text_matrix)

    # print(lst)
    lst = np.array(lst)

    # print(lst)
    similarities = []
    for similarity in lst:
        cosine_similarities = cosine_similarity(similarity, tfidf)
        #cosine_similarities = cosine_similarities[0]
        cosine_similarities = (-cosine_similarities).argsort()
        print(cosine_similarities)
        similarities.extend(cosine_similarities)
        #print(similarities)
    similarities = np.array(similarities)
    print(similarities)
    similarities = similarities.flatten(order='F')
    print(similarities)
    #sum(similarities, [])

    #print(similarities)

    #similarities = np.array(similarities)
    #print(similarities.dtype)
    #similarities = (-similarities).argsort()
    #print(similarities)

    # songs = []
    #
    # songs.append(lyrics.iloc[similarities[3]])
    # songs.append(lyrics.iloc[similarities[1]])
    # songs.append(lyrics.iloc[similarities[2]])
    #
    # # for song in similarities[:20]:
    # #     songs.append(lyrics.iloc[song])
    # print(songs)

    # for array in similarities:
    #
    #     sorted_index=array.argsort()[0][::-1]
    #
    #     five_most_similar_text=[lyrics.iloc[i]['title'] for i in sorted_index[1:5]]
    #
    #     most_similar_text[sorted_index[0]]=five_most_similar_text

    return similarities